<a href="https://colab.research.google.com/github/seelokyen/bottledetectionCNNmodel/blob/main/CP2_Train_CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon May 19 12:27:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/MyDrive/bottles_dataset/data.zip /content

Mounted at /content/gdrive


In [3]:
# Unzip images to a custom data folder
!unzip -q /content/data.zip -d /content/custom_data

In [4]:
import os
import random
import shutil

# Set paths
image_dir = '/content/custom_data/data.zip/images'
label_dir = '/content/custom_data/data.zip/labels'
output_base = '/content/sampled_10_percent'

# Create output folders
os.makedirs(f'{output_base}/images', exist_ok=True)
os.makedirs(f'{output_base}/labels', exist_ok=True)

# Get image filenames
image_files = [f for f in os.listdir(image_dir)]

# Select 10%
sample_size = int(0.10 * len(image_files))
sampled_images = random.sample(image_files, sample_size)

# Copy images and labels
for img_file in sampled_images:
    label_file = os.path.splitext(img_file)[0] + '.txt'
    shutil.copy(os.path.join(image_dir, img_file), f'{output_base}/images/{img_file}')
    shutil.copy(os.path.join(label_dir, label_file), f'{output_base}/labels/{label_file}')

print(f"Copied {sample_size} images and labels to {output_base}")


Copied 1434 images and labels to /content/sampled_10_percent


In [5]:
# Define split percentages
train_pct, val_pct, test_pct = 0.7, 0.15, 0.15

# Setup directories
base = '/content/sampled_10_percent'
splits = ['train', 'val', 'test']
for split in splits:
    os.makedirs(f'{base}/{split}/images', exist_ok=True)
    os.makedirs(f'{base}/{split}/labels', exist_ok=True)

# Shuffle again and split
random.shuffle(sampled_images)
total = len(sampled_images)
train_end = int(train_pct * total)
val_end = train_end + int(val_pct * total)

# Assign to splits
for i, img_file in enumerate(sampled_images):
    split = 'train' if i < train_end else 'val' if i < val_end else 'test'
    label_file = os.path.splitext(img_file)[0] + '.txt'
    shutil.copy(os.path.join(base, 'images', img_file), f'{base}/{split}/images/{img_file}')
    shutil.copy(os.path.join(base, 'labels', label_file), f'{base}/{split}/labels/{label_file}')

print("Finished dataset splitting into train/val/test")

Finished dataset splitting into train/val/test


In [6]:
# Create data.yaml file
data_yaml = """
path: /content/sampled_10_percent
train: train/images
val: val/images
test: test/images

nc: 1
names: ['bottle']
"""

with open('/content/sampled_10_percent/data.yaml', 'w') as f:
    f.write(data_yaml)

print("Finished creating data.yaml file")

Finished creating data.yaml file


In [7]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [8]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # or yolov8s.pt for slightly larger

model.train(
    data='/content/sampled_10_percent/data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    name='baseline_yolov8',
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 90.0MB/s]


Ultralytics 8.3.139 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/sampled_10_percent/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=baseline_yolov8, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, po

100%|██████████| 755k/755k [00:00<00:00, 25.2MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all        215        907      0.909      0.875      0.933      0.697



50 epochs completed in 0.270 hours.
Optimizer stripped from runs/detect/baseline_yolov8/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/baseline_yolov8/weights/best.pt, 6.2MB

Validating runs/detect/baseline_yolov8/weights/best.pt...
Ultralytics 8.3.139 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:03<00:00,  1.83it/s]


                   all        215        907       0.91      0.875      0.933      0.697
Speed: 0.3ms preprocess, 2.3ms inference, 0.0ms loss, 4.1ms postprocess per image
Results saved to runs/detect/baseline_yolov8


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ae6eee8fb50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [9]:
metrics = model.val()

Ultralytics 8.3.139 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 909.5±352.2 MB/s, size: 43.3 KB)


val: Scanning /content/sampled_10_percent/val/labels.cache... 215 images, 0 backgrounds, 0 corrupt: 100%|██████████| 215/215 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 42, len(boxes) = 907. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:04<00:00,  2.85it/s]


                   all        215        907      0.909      0.875      0.933      0.698
Speed: 1.8ms preprocess, 6.0ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs/detect/baseline_yolov82


In [10]:
model.predict(source='/content/sampled_10_percent/test/images', save=True)


image 1/216 /content/sampled_10_percent/test/images/-219_jpg.rf.63bba48c5c317e9f1b1a14a1c643efbe.jpg: 640x640 3 bottles, 9.5ms
image 2/216 /content/sampled_10_percent/test/images/-227_jpg.rf.ec8c93c1f24ee9ef96adab4f53924b22.jpg: 640x640 2 bottles, 8.6ms
image 3/216 /content/sampled_10_percent/test/images/-234_jpg.rf.e7b465f9b6a186630e01072ec712b4c0.jpg: 640x640 2 bottles, 8.3ms
image 4/216 /content/sampled_10_percent/test/images/-376_jpg.rf.0bb7bbf99a1690b8e33203ea8ad7ddeb.jpg: 640x640 1 bottle, 8.2ms
image 5/216 /content/sampled_10_percent/test/images/-376_jpg.rf.2095f27e941f69ec5cc40f2ec19a337f.jpg: 640x640 1 bottle, 8.5ms
image 6/216 /content/sampled_10_percent/test/images/-38_jpg.rf.a955158cdbdfd326137a1baa6a6c735e.jpg: 640x640 1 bottle, 10.4ms
image 7/216 /content/sampled_10_percent/test/images/-7_jpg.rf.e49026eb4564812a75c60b26632713b7.jpg: 640x640 4 bottles, 13.2ms
image 8/216 /content/sampled_10_percent/test/images/000022_jpg.rf.7bb7fd36e6a56e864b41db5e4f970bd4.jpg: 640x640 5 

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'bottle'}
 obb: None
 orig_img: array([[[  8,   7,   9],
         [  0,   0,   1],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[255, 254, 255],
         [243, 242, 244],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[252, 251, 253],
         [255, 254, 255],
         [241, 241, 241],
         ...,
         [253, 253, 253],
         [253, 253, 253],
         [253, 253, 253]],
 
        ...,
 
        [[255, 255, 248],
         [255, 254, 245],
         [254, 251, 243],
         ...,
         [253, 253, 253],
         [247, 247, 247],
         [241, 241, 241]],
 
        [[255, 255, 248],
         [255, 255, 248],
         [255, 254, 245],
         ...,
         [236, 236